## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-06-02-44-13 +0000,48hills,Supes move forward $40 million tax break for l...,https://48hills.org/2026/02/supes-move-forward...
1,2026-02-06-02-42-40 +0000,nypost,LA politician stumped on long-awaited homeless...,https://nypost.com/2026/02/05/us-news/la-polit...
2,2026-02-06-02-27-39 +0000,nyt,"To Win Back the House, Democrats Take the Figh...",https://www.nytimes.com/2026/02/05/us/politics...
3,2026-02-06-02-26-28 +0000,nyt,"Uber Found Liable in Rape by Driver, Setting S...",https://www.nytimes.com/2026/02/05/business/ub...
4,2026-02-06-02-19-00 +0000,wsj,The Famous Names in the Latest Release of Epst...,https://www.wsj.com/us-news/law/what-we-know-s...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2521/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
98,trump,27
42,epstein,22
130,new,15
281,ai,13
44,california,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
94,2026-02-05-22-38-00 +0000,wsj,"Under a new policy, the Trump administration i...",https://www.wsj.com/politics/policy/trump-admi...,88
103,2026-02-05-22-22-03 +0000,nypost,Hillary Clinton demands public hearing on Epst...,https://nypost.com/2026/02/05/us-news/hillary-...,69
23,2026-02-06-01-19-00 +0000,wsj,Trump Calls for New Nuclear Pact With Russia a...,https://www.wsj.com/politics/national-security...,68
200,2026-02-05-17-50-42 +0000,nyt,LA Olympics Chief Keeps Low Profile in Milan G...,https://www.nytimes.com/2026/02/05/world/europ...,61
46,2026-02-06-00-25-42 +0000,nypost,Match.com sent Jeffrey Epstein dating profiles...,https://nypost.com/2026/02/05/us-news/doj-file...,60


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
94,88,2026-02-05-22-38-00 +0000,wsj,"Under a new policy, the Trump administration i...",https://www.wsj.com/politics/policy/trump-admi...
200,61,2026-02-05-17-50-42 +0000,nyt,LA Olympics Chief Keeps Low Profile in Milan G...,https://www.nytimes.com/2026/02/05/world/europ...
217,50,2026-02-05-17-15-17 +0000,nypost,"Stocks plunge on AI spending fears, unemployme...",https://nypost.com/2026/02/05/business/stocks-...
86,46,2026-02-05-22-55-54 +0000,nyt,Los Angeles Civic Leader Austin Beutner Drops ...,https://www.nytimes.com/2026/02/05/us/los-ange...
73,46,2026-02-05-23-29-46 +0000,nypost,"Russia, Ukraine exchange 300 prisoners as US p...",https://nypost.com/2026/02/05/world-news/russi...
20,39,2026-02-06-01-32-17 +0000,nypost,Nancy Guthrie’s neighbor reported suspicious w...,https://nypost.com/2026/02/05/us-news/nancy-gu...
71,35,2026-02-05-23-37-02 +0000,startribune,"In rare move, this northern Minnesota county i...",https://www.startribune.com/in-rare-move-this-...
251,35,2026-02-05-15-21-45 +0000,cbc,"Goodbye EV sales mandate, hello purchase rebat...",https://www.cbc.ca/news/politics/carney-droppi...
89,34,2026-02-05-22-47-33 +0000,latimes,California bill would make fossil fuel compani...,https://www.latimes.com/business/story/2026-02...
343,32,2026-02-05-05-11-00 +0000,wsj,Democrats’ drive to reach a deal to put new re...,https://www.wsj.com/politics/policy/congress-e...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
